In [1]:
# Part 1: Import Libraries and Define the Cage Class
import copy
import random

class Cage:
    def __init__(self, cells, operation, target):
        self.cells = cells
        self.operation = operation
        self.target = target


In [2]:
# Part 2: Function to Generate KenKen Puzzle

def generate_KenKen(size):
    gameboard = [[0] * size for _ in range(size)]
    return gameboard
    # raise NotImplementedError


# Part 3: Function to Fill Grid with Numbers

def fill_grid_with_numbers(grid, size):
    """
    This function fills the grid with numbers 1 to size such that
    each row and each column has unique values. It uses backtracking
    to ensure all cells are filled with valid numbers.
    """

    def is_safe_to_place(grid, row, col, num):
        # Check row and column uniqueness
        curr_row = grid[row]
        curr_col = [grid[i][col] for i in range(size)]

        if num in curr_row or num in curr_col:
            return False

        return True

        # raise NotImplementedError


    def fill_grid_backtracking(grid, row, col):
        # Fills the grid recursively with valid numbers using backtracking

        for num in range(1, size + 1):
            if is_safe_to_place(grid, row, col, num):
                grid[row][col] = num

                new_row = (row + 1) % size
                new_col = col + 1 if row + 1 == size else col

                if new_row == size or new_col == size:
                    return True

                if fill_grid_backtracking(grid, new_row, new_col):
                    return True
                else:
                    grid[row][col] = 0

        return False
        # raise NotImplementedError

    # Start backtracking from the first cell
    fill_grid_backtracking(grid, 0, 0)



# Part 4: Function to Generate Random Cages

def generate_random_cages(grid, size):
    cages = []
    visited = [[False] * size for _ in range(size)]

    for i in range(size):
        for j in range(size):
            if not visited[i][j]:
                cage_size = random.randint(1, size)
                cells = [(i, j)]
                visited[i][j] = True

                while len(cells) < cage_size:
                    x, y = cells[-1]
                    neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]
                                 if 0 <= x + dx < size and 0 <= y + dy < size and not visited[x + dx][y + dy]]
                    if neighbors:
                        next_cell = random.choice(neighbors)
                        cells.append(next_cell)
                        visited[next_cell[0]][next_cell[1]] = True
                    else:
                        break

                operation = random.choice(['+', '*']) if cage_size > 2 else random.choice(['+', '-', '*', '/'])
                
                # print(cells)
                # print(operation)
                
                target = calculate_target(grid, cells, operation)
                cages.append(Cage(cells, operation, target))

    return cages


# Function to calculate the target based on the operation for a given cage
def calculate_target(grid, cells, operation):
    match operation:
        case '+':
            result = 0
            for cell in cells:
                num = grid[cell[0]][cell[1]]
                result += num
        
        case '-':
            cell1 = cells[0]
            if len(cells) == 1:
                return grid[cell1[0]][cell1[1]]
            cell2 = cells[1]

            num1 = grid[cell1[0]][cell1[1]]
            num2 = grid[cell2[0]][cell2[1]]

            result = abs(num1 - num2)
        
        case '*':
            result = 1
            for cell in cells:
                num = grid[cell[0]][cell[1]]
                result *= num
        
        case '/':
            cell1 = cells[0]
            if len(cells) == 1:
                return grid[cell1[0]][cell1[1]]
            cell2 = cells[1]

            num1 = grid[cell1[0]][cell1[1]]
            num2 = grid[cell2[0]][cell2[1]]

            if num1 >= num2:
                result = num1 / num2
            else:
                result = num2 / num1

    return result

    # raise NotImplementedError




In [3]:
# Part 5: KenKen Backtracking Solver

def solve_kenken(grid, cages):
    
    size = len(grid)

    def solve_kenken_backtracking(row, col):
        for num in range(1, size + 1):
            if is_safe_kenken(grid, row, col, num, cages):
                grid[row][col] = num

                new_row = (row + 1) % size
                new_col = col + 1 if row + 1 == size else col

                if new_row == size or new_col == size:
                    return True

                if solve_kenken_backtracking(new_row, new_col):
                    return True
                else:
                    grid[row][col] = 0

        return False

    solve_kenken_backtracking(0, 0)
    # raise NotImplementedError


def is_safe_kenken(grid, row, col, num, cages):
    # Check row, column and cage numbers uniqueness
    size = len(grid)

    curr_row = grid[row]
    curr_col = [grid[i][col] for i in range(size)]

    if num in curr_row or num in curr_col:
        return False

    grid[row][col] = num

    cage_cell = row, col
    for cage in cages:
        if cage_cell in cage.cells:
            values = [grid[i][j] for i, j in cage.cells]
            if not validate_cage_operation(cage.operation, values, cage.target):
                grid[row][col] = 0
                return False

    return True

    # raise NotImplementedError


def validate_cage_operation(operation, values, target):
    # Check if the operation on the cage values matches the target

    match operation:
        case '+':
            return sum(values) <= target

        case '-':
            if len(values) == 1:
                return values[0] == target
            return abs(values[0] - values[1]) == target if values[0] != 0 and values[1] != 0 else True

        case '*':
            res = 1
            for num in values:
                res *= num if num != 0 else 1
            return res <= target

        case '/':
            if len(values) == 1:
                return values[0] == target
            return max(values) / min(values) if values[0] != 0 and values[1] != 0 else True

    # raise NotImplementedError


def find_unassigned_location(grid):
    # Find the first empty cell in the grid
    pass
    # raise NotImplementedError



In [4]:
# Part 6: KenKen Domain Constraint Solver

def solve_kenken_csp(grid, cages):
    """
    Function to solve the KenKen grid using Constraint Satisfaction Problem (CSP)
    """

    size = len(grid)
    def create_domains(grid):
        """
        Function to create domains for each cell in the grid
        """
        domains = {(i, j): [i for i in range(1, size + 1)] for i in range(size) for j in range(size)}
        return domains


    def is_valid_assignment(i, j, val, assignment):
        """
        Function to check if assigning a value to a cell is valid
        """
        domains_dic = copy.deepcopy(assignment)

        curr_row_locations = [(i, idx) for idx in range(size)]
        curr_col_locations = [(idx, j) for idx in range(size)]

        # Checking for conflicts in the same row
        for cell_loc in curr_row_locations:
            if cell_loc == (i, j):
                continue
            domains_dic[cell_loc].remove(val)
            if len(domains_dic[cell_loc]) == 0:
                return False


        # Checking for conflicts in the same column
        for cell_loc in curr_col_locations:
            if cell_loc == (i, j):
                continue
            domains_dic[cell_loc].remove(val)
            if len(domains_dic[cell_loc]) == 0:
                return False


        # Checking for conflicts in the related cage
        curr_cell = i, j
        for cage in cages:
            if curr_cell in cage.cells:
                values = [grid[ii][jj] for ii, jj in cage.cells]
                values.remove(grid[i][j])
                values.append(val)
                if not validate_cage_operation(cage.operation, values, val):
                    return False

        assignment = domains_dic
        return True


    def validate_cage_operation(operation, values, target):
        # Check if the operation on the cage values matches the target

        match operation:
            case '+':
                return sum(values) <= target

            case '-':
                if len(values) == 1:
                    return values[0] == target
                return abs(values[0] - values[1]) == target if values[0] != 0 and values[1] != 0 else True

            case '*':
                res = 1
                for num in values:
                    res *= num if num != 0 else 1
                return res <= target

            case '/':
                if len(values) == 1:
                    return values[0] == target
                return max(values) / min(values) if values[0] != 0 and values[1] != 0 else True

        # raise NotImplementedError


    def find_unassigned_location(assignment):
        """
        Function to find an unassigned location in the grid
        """
        cell_location = None

        cell_location = None
        min_len = float('inf')

        for key, value in assignment.items():
            i, j = key
            if grid[i][j] == 0 and len(value) < min_len:
                cell_location = key
                min_len = len(value)

        if cell_location == None:
            return "SHALQAM"

        return cell_location


    def solve_csp(assignment):
        """
        Recursive function to solve grid with CSP
        """
        unassigned_loc = find_unassigned_location(assignment)
        
        if unassigned_loc == "SHALQAM":
            return True
        
        for item in assignment[unassigned_loc]:
            # grid[unassigned_loc[0]][unassigned_loc[1]] = item
            if is_valid_assignment(unassigned_loc[0], unassigned_loc[1], item, assignment):
                assignment_copy = copy.deepcopy(assignment)
                solve_csp(assignment_copy)
            else:
                grid[unassigned_loc[0]][unassigned_loc[1]] = 0
                return False

        return True


    # Create initial domains for each cell in the grid
    domains = create_domains(grid)

    # print(domains)

    assignment = {(i, j): val for (i, j), val in domains.items()}

    # Solve the KenKen grid using CSP
    if solve_csp(assignment):
        solved_grid = [[assignment[(i, j)] for j in range(len(grid))] for i in range(len(grid))]
        return solved_grid
    else:
        return None


In [15]:
mat = generate_KenKen(3)
fill_grid_with_numbers(mat, 3)
cages = generate_random_cages(mat, 3)
mat = generate_KenKen(3)



solve_kenken_csp(mat, cages)
mat

RecursionError: maximum recursion depth exceeded

In [9]:
# Part 7: Print Solution

def print_solution(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))


In [34]:
# Part 8: Run Example

size = 5
unsolved_grid, kenken_cages = generate_KenKen(size)

print("Generated KenKen Cages:")
for cage in kenken_cages:
    print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target}")
print("-----------------------")

# Solve using Backtracking solver
backtrack_grid = copy.deepcopy(unsolved_grid)
if solve_kenken(backtrack_grid, kenken_cages):
    print("Solved KenKen Puzzle (BackTracking):")
    print_solution(backtrack_grid)
else:
    print("No solution found using BackTracking.")
print("-----------------------")


# Solve using domain CSP solver
csp_solved, steps = solve_kenken_csp(unsolved_grid, kenken_cages)
if csp_solved:
    print("Solved KenKen Puzzle (domain CSP):")
    print_solution(csp_solved, steps)
else:
    print("No solution found using CSP.")


ValueError: too many values to unpack (expected 2)